# LEGO Amazon Analysis (Medium Notebook)
This notebook loads the uploaded data files, performs basic cleaning and exploratory data analysis (EDA), and saves a set of images you can use in your presentation. It is designed to be beginner-friendly but also provide useful charts and insights for the project.

**Files used in this notebook**:
- `/mnt/data/geoMap.csv` (uploaded)
- `/mnt/data/toy-products-on-amazon-metadata.json` (uploaded)

If additional CSV data (e.g., the full Amazon dataset) is available, place it in the `/mnt/data` folder and re-run the cells.

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

print('pandas:', pd.__version__)
print('numpy:', np.__version__)
print('matplotlib:', plt.matplotlib.__version__)

pandas: 2.2.2
numpy: 2.0.2
matplotlib: 3.10.0


In [42]:
from google.colab import files

uploaded = files.upload()


Saving toy-products-on-amazon-metadata.json to toy-products-on-amazon-metadata (1).json
Saving geoMap.csv to geoMap (1).csv


In [43]:
import os

print(os.listdir())


['.config', 'geoMap (1).csv', 'amazon_co-ecommerce_sample[1].csv', 'toy-products-on-amazon-metadata (1).json', 'toy-products-on-amazon-metadata.json', 'archive (4).zip', 'geoMap.csv', 'sample_data']


In [44]:
import pandas as pd

geo_map = pd.read_csv('geoMap.csv')
print(geo_map.head())  # Shows first few rows


                     Category: All categories
Country           LEGO: (11/28/20 - 11/28/25)
Christmas Island                          NaN
Poland                                    100
Hungary                                    94
Czechia                                    93


In [47]:
import json

with open('toy-products-on-amazon-metadata.json') as f:
    toy_data = json.load(f)

# Check the type and top-level keys
print(type(toy_data))
print(toy_data.keys())


<class 'dict'>
dict_keys(['@context', 'alternateName', 'conformsTo', 'license', 'distribution', 'recordSet', 'version', 'keywords', 'isAccessibleForFree', 'includedInDataCatalog', 'creator', 'publisher', 'thumbnailUrl', 'dateModified', 'datePublished', '@type', 'name', 'url', 'description'])


In [48]:
# Check distribution
print(toy_data['distribution'])

# Check recordSet
print(toy_data['recordSet'])



[{'contentUrl': 'https://www.kaggle.com/api/v1/datasets/download/PromptCloudHQ/toy-products-on-amazon?datasetVersionNumber=1', 'contentSize': '8.179 MB', 'md5': '9Ei/TRAqvpe0f6Sn32UPCg==', 'encodingFormat': 'application/zip', '@id': 'archive.zip', '@type': 'cr:FileObject', 'name': 'archive.zip', 'description': 'Archive containing all the contents of the Toy Products on Amazon dataset'}, {'contentUrl': 'amazon_co-ecommerce_sample.csv', 'containedIn': {'@id': 'archive.zip'}, 'encodingFormat': 'text/csv', '@id': 'amazon_co-ecommerce_sample.csv_fileobject', '@type': 'cr:FileObject', 'name': 'amazon_co-ecommerce_sample.csv', 'description': 'This is a pre-crawled dataset, taken as subset of a bigger [**dataset (more than 115k products)**][1] that was created by extracting data from Amazon.com.\n\n  [1]: https://www.promptcloud.com/datastock-access-ready-to-use-datasets/?utm_source=kaggle&utm_medium=referral&utm_campaign=amazon'}]
[{'field': [{'dataType': ['sc:Text'], 'source': {'fileObject':

In [49]:
import pandas as pd

toy_items = toy_data['recordSet']  # Adjust if distribution contains data instead
toy_df = pd.json_normalize(toy_items)
print(toy_df.head())


                                               field  \
0  [{'dataType': ['sc:Text'], 'source': {'fileObj...   

                              @id         @type  \
0  amazon_co-ecommerce_sample.csv  cr:RecordSet   

                             name  \
0  amazon_co-ecommerce_sample.csv   

                                         description  
0  This is a pre-crawled dataset, taken as subset...  


In [50]:
import pandas as pd

# Get the recordSet
record = toy_data['recordSet'][0]  # first (and only) item

# Extract the 'field' key
fields = record['field']
print(fields[0])  # Inspect first product field


{'dataType': ['sc:Text'], 'source': {'fileObject': {'@id': 'amazon_co-ecommerce_sample.csv_fileobject'}, 'extract': {'column': 'uniq_id'}}, '@id': 'amazon_co-ecommerce_sample.csv/uniq_id', '@type': 'cr:Field', 'name': 'uniq_id', 'description': 'The unique user ID given by the website'}


In [51]:
toy_df = pd.json_normalize(fields)
print(toy_df.head())


    dataType                                                @id     @type  \
0  [sc:Text]             amazon_co-ecommerce_sample.csv/uniq_id  cr:Field   
1  [sc:Text]        amazon_co-ecommerce_sample.csv/product_name  cr:Field   
2  [sc:Text]        amazon_co-ecommerce_sample.csv/manufacturer  cr:Field   
3  [sc:Text]               amazon_co-ecommerce_sample.csv/price  cr:Field   
4  [sc:Text]  amazon_co-ecommerce_sample.csv/number_availabl...  cr:Field   

                        name                              description  \
0                    uniq_id  The unique user ID given by the website   
1               product_name                         The product name   
2               manufacturer                    The manufacturer name   
3                      price                 The price of the product   
4  number_available_in_stock         The number of products available   

                       source.fileObject.@id      source.extract.column  
0  amazon_co-ecommerce_s

In [52]:
print(toy_data['distribution'])


[{'contentUrl': 'https://www.kaggle.com/api/v1/datasets/download/PromptCloudHQ/toy-products-on-amazon?datasetVersionNumber=1', 'contentSize': '8.179 MB', 'md5': '9Ei/TRAqvpe0f6Sn32UPCg==', 'encodingFormat': 'application/zip', '@id': 'archive.zip', '@type': 'cr:FileObject', 'name': 'archive.zip', 'description': 'Archive containing all the contents of the Toy Products on Amazon dataset'}, {'contentUrl': 'amazon_co-ecommerce_sample.csv', 'containedIn': {'@id': 'archive.zip'}, 'encodingFormat': 'text/csv', '@id': 'amazon_co-ecommerce_sample.csv_fileobject', '@type': 'cr:FileObject', 'name': 'amazon_co-ecommerce_sample.csv', 'description': 'This is a pre-crawled dataset, taken as subset of a bigger [**dataset (more than 115k products)**][1] that was created by extracting data from Amazon.com.\n\n  [1]: https://www.promptcloud.com/datastock-access-ready-to-use-datasets/?utm_source=kaggle&utm_medium=referral&utm_campaign=amazon'}]


In [53]:
from google.colab import files

uploaded = files.upload()


Saving amazon_co-ecommerce_sample[1].csv to amazon_co-ecommerce_sample[1] (1).csv


In [54]:
import os

print(os.listdir())  # You should see your CSV file here


['.config', 'geoMap (1).csv', 'amazon_co-ecommerce_sample[1] (1).csv', 'amazon_co-ecommerce_sample[1].csv', 'toy-products-on-amazon-metadata (1).json', 'toy-products-on-amazon-metadata.json', 'archive (4).zip', 'geoMap.csv', 'sample_data']


In [56]:
import os

print(os.listdir())


['.config', 'geoMap (1).csv', 'amazon_co-ecommerce_sample[1] (1).csv', 'amazon_co-ecommerce_sample[1].csv', 'toy-products-on-amazon-metadata (1).json', 'toy-products-on-amazon-metadata.json', 'archive (4).zip', 'geoMap.csv', 'sample_data']


In [57]:
import pandas as pd

df = pd.read_csv('amazon_co-ecommerce_sample[1].csv')
print(df.head())


                            uniq_id  \
0  eac7efa5dbd3d667f26eb3d3ab504464   
1  b17540ef7e86e461d37f3ae58b7b72ac   
2  348f344247b0c1a935b1223072ef9d8a   
3  e12b92dbb8eaee78b22965d2a9bbbd9f   
4  e33a9adeed5f36840ccc227db4682a36   

                                        product_name manufacturer   price  \
0                              Hornby 2014 Catalogue       Hornby   £3.42   
1  FunkyBuys® Large Christmas Holiday Express Fes...    FunkyBuys  £16.99   
2  CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...          ccf   £9.99   
3     HORNBY Coach R4410A BR Hawksworth Corridor 3rd       Hornby  £39.99   
4  Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...       Hornby  £32.19   

  number_available_in_stock number_of_reviews  number_of_answered_questions  \
0                     5 new                15                           1.0   
1                       NaN                 2                           1.0   
2                     2 new                17                    

In [58]:
# Check columns and types
print(df.columns)
print(df.info())

# Check first few rows
print(df.head())


Index(['uniq_id', 'product_name', 'manufacturer', 'price',
       'number_available_in_stock', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
       'amazon_category_and_sub_category',
       'customers_who_bought_this_item_also_bought', 'description',
       'product_information', 'product_description',
       'items_customers_buy_after_viewing_this_item',
       'customer_questions_and_answers', 'customer_reviews', 'sellers'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 9993 non-null   object 
 3   price                   

In [60]:
print(df.columns.tolist())



['uniq_id', 'product_name', 'manufacturer', 'price', 'number_available_in_stock', 'number_of_reviews', 'number_of_answered_questions', 'average_review_rating', 'amazon_category_and_sub_category', 'customers_who_bought_this_item_also_bought', 'description', 'product_information', 'product_description', 'items_customers_buy_after_viewing_this_item', 'customer_questions_and_answers', 'customer_reviews', 'sellers']


In [61]:
print(df.head())        # First 5 rows
print(df.columns.tolist())  # All column names
print(df.info())        # Data types and missing values


                            uniq_id  \
0  eac7efa5dbd3d667f26eb3d3ab504464   
1  b17540ef7e86e461d37f3ae58b7b72ac   
2  348f344247b0c1a935b1223072ef9d8a   
3  e12b92dbb8eaee78b22965d2a9bbbd9f   
4  e33a9adeed5f36840ccc227db4682a36   

                                        product_name manufacturer   price  \
0                              Hornby 2014 Catalogue       Hornby   £3.42   
1  FunkyBuys® Large Christmas Holiday Express Fes...    FunkyBuys  £16.99   
2  CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...          ccf   £9.99   
3     HORNBY Coach R4410A BR Hawksworth Corridor 3rd       Hornby  £39.99   
4  Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...       Hornby  £32.19   

  number_available_in_stock number_of_reviews  number_of_answered_questions  \
0                     5 new                15                           1.0   
1                       NaN                 2                           1.0   
2                     2 new                17                    

In [62]:
import pandas as pd

# Extract numeric rating
df['rating'] = df['average_review_rating'].str.extract(r'([\d\.]+)').astype(float)
print(df['rating'].head())


0    4.9
1    4.5
2    3.9
3    5.0
4    4.7
Name: rating, dtype: float64


In [64]:
# Use raw string for regex and include £ symbol
df['price'] = df['price'].replace(r'[£$,]', '', regex=True)

# Convert to float
df['price'] = df['price'].astype(float)

print(df['price'].head())


ValueError: could not convert string to float: '269.00 - 699.99'

In [65]:
def parse_price(x):
    try:
        # Remove currency symbols and spaces
        x = x.replace('£','').replace('$','').replace(',','').strip()
        # If it’s a range like "269.00 - 699.99", take average
        if '-' in x:
            parts = x.split('-')
            return (float(parts[0]) + float(parts[1])) / 2
        else:
            return float(x)
    except:
        return None  # return NaN if conversion fails


In [66]:
df['price'] = df['price'].apply(parse_price)
print(df['price'].head())


0     3.42
1    16.99
2     9.99
3    39.99
4    32.19
Name: price, dtype: float64


In [67]:
# Split category
df[['main_category','sub_category']] = df['amazon_category_and_sub_category'].str.split('>', n=1, expand=True)
df['main_category'] = df['main_category'].str.strip()
df['sub_category'] = df['sub_category'].str.strip()

print(df[['main_category','sub_category']].head())


  main_category                                       sub_category
0       Hobbies  Model Trains & Railway Sets > Rail Vehicles > ...
1       Hobbies  Model Trains & Railway Sets > Rail Vehicles > ...
2       Hobbies  Model Trains & Railway Sets > Rail Vehicles > ...
3       Hobbies  Model Trains & Railway Sets > Rail Vehicles > ...
4       Hobbies  Model Trains & Railway Sets > Rail Vehicles > ...


In [68]:
df['customers_who_bought_this_item_also_bought'].fillna('None', inplace=True)
df['items_customers_buy_after_viewing_this_item'].fillna('None', inplace=True)
df['customer_reviews'].fillna('None', inplace=True)


/tmp/ipython-input-2593417961.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['customers_who_bought_this_item_also_bought'].fillna('None', inplace=True)
/tmp/ipython-input-2593417961.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(

In [69]:
top_products = df.sort_values(by='rating', ascending=False).head(10)
print(top_products[['product_name','rating','price']])


                                           product_name  rating   price
9998  Justice League of America Series 3 Green Lante...     5.0   49.81
9997        Defiance Lawkeeper Metal Badge Prop Replica     5.0   43.99
9995  Batman 1966 TV Series Action Figures - The Rid...     5.0   22.95
9992  Star Wars The Clone Wars CW01 Captain Rex 3.75...     5.0   32.99
3        HORNBY Coach R4410A BR Hawksworth Corridor 3rd     5.0   39.99
9974  Lavender Brown Character Wand. Harry Potter No...     5.0   26.99
9973  Hot Toys - Batman The Dark Knight Rises Movie ...     5.0  399.99
9972                      Funko POP! Harry Potter Sword     5.0   13.90
9971             Star Wars 30th #04 R2-D2 Action Figure     5.0   22.69
9970        Star Trek Play Arts Kai Spock Action Figure     5.0   49.88


In [70]:
avg_price = df.groupby('main_category')['price'].mean()
print(avg_price)


main_category
Arts & Crafts                        6.535061
Baby & Toddler Toys                 26.972338
Bags                                 7.467500
Bedding & Linens                     7.490000
Camping & Hiking                     4.150000
Car Parts                                 NaN
Characters & Brands                 21.976725
Cooking & Dining                     4.862500
Die-Cast & Toy Vehicles             26.928547
Dogs                                24.985000
Dolls & Accessories                 11.523175
Educational Toys                    22.300000
Electronic Toys                     29.355000
Fancy Dress                         13.546033
Figures & Playsets                  26.573303
Games                               15.303797
Gardening                            5.480000
Handbags & Shoulder Bags             3.990000
Hobbies                             36.140776
Home Accessories                     8.978000
Indoor Lighting                      8.990000
Jams, Honey & Spread

In [71]:
missing_reviews = df['customer_reviews'].isna().sum()
print(f"Products missing reviews: {missing_reviews}")


Products missing reviews: 0
